In [1]:
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
import time
import os
import matplotlib.pylab as plt
from torch.optim import Adam
import numpy as np
import wandb
import gc

/home/user/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip nature_12K.zip

In [5]:
training_data_path = "inaturalist_12K/train/"

In [6]:
class CNN(nn.Module):
    def __init__(
        self,
        input_dimension:tuple,
        number_of_filters: int,
        filter_size: tuple,
        stride: int,
        padding: int,
        max_pooling_size: tuple,
        n_neurons: int,
        n_classes: int,
        conv_activation: nn.Module,
        dense_activation: nn.Module,
        dropout_rate:float,
        use_batchnorm:bool,
        factor:float,
        dropout_organisation:int
    ):
        super().__init__()
        self.conv_blocks = nn.ModuleList([])
        in_c = input_dimension[0]
        for i in range(0, 5):
            add_dropout = i%dropout_organisation > 0
            out_c = int((factor**i)*number_of_filters)
            if out_c<=0:
                out_c = 3
            conv_block = self.create_conv_block(in_c, out_c, filter_size, max_pooling_size, stride, padding, conv_activation, dropout_rate, use_batchnorm, add_dropout)
            self.conv_blocks.append(conv_block)
            in_c = out_c

        self.flatten = nn.Flatten()
        
        r = torch.ones(1, *input_dimension)
        for block in self.conv_blocks:
            block.eval()
            r = block(r)
        in_features = int(np.prod(r.size()[1:]))

        self.dense_block1 = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=n_neurons),
            dense_activation,
            nn.Linear(in_features=n_neurons, out_features=n_classes),
            nn.LogSoftmax(dim=1)
        )

    def create_conv_block(self, in_c, out_c, kernel_size, max_pooling_size, stride, padding, conv_activation, dropout_rate, use_batchnorm, add_dropout):
        layers = [
            nn.Conv2d(in_c, out_c, kernel_size=kernel_size, stride=stride, padding=padding),
            conv_activation,
        ]
        if use_batchnorm:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.MaxPool2d(kernel_size=max_pooling_size))
        if add_dropout:
            layers.append(nn.Dropout(p=dropout_rate))
        return nn.Sequential(*layers)


    def __call__(self, x):
        r = x
        for block in self.conv_blocks:
            r = block(r)
        r = self.flatten(r)
        output = self.dense_block1(r)
        return output

In [7]:
def get_transform(use_augmentation):
    if use_augmentation:
        transform = transforms.Compose([
            transforms.RandomCrop(50, padding=1),
            transforms.RandomGrayscale(p=0.1),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=(0, 20)),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225],
                                    inplace=False),
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
    return transform

In [8]:
def train(config):
    
    training_data = ImageFolder(root=training_data_path, transform=get_transform(config['use_augmentation']))
    train_size = int(0.8 * len(training_data))
    val_size = len(training_data) - train_size
    train_set, validation_set = random_split(training_data, [train_size, val_size])
    train_dataloader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(validation_set, batch_size=config['batch_size'], shuffle=False)


    training_loss, training_accuracy, validation_loss, validation_accuracy = [],  [],  [], []

    activations = {
      'relu': nn.ReLU(),
      'gelu': nn.GELU(),
      'silu': nn.SiLU(),
      'mish': nn.Mish(),
      'relu6':nn.ReLU6(),
      'tanh': nn.Tanh(),
      'sigmoid': nn.Sigmoid()
    }
    torch.cuda.empty_cache()
    model = CNN(
      input_dimension=(3,224,224),
        number_of_filters=config['number_of_filters'],
        filter_size =(config['filter_size'],config['filter_size']),
        stride=config['stride'],
        padding=config['padding'],
        max_pooling_size=(config['max_pooling_size'],config['max_pooling_size']),
        n_neurons=config['n_neurons'],
        n_classes=config['n_classes'],
        conv_activation=activations[config['conv_activation']],
        dense_activation=activations[config['dense_activation']],
        dropout_rate=config['dropout_rate'],
        use_batchnorm=config['use_batchnorm'],
        factor=config['factor'],
        dropout_organisation=config['dropout_organisation']
    ).to(device)
    optimizer = Adam(model.parameters(), lr = config['learning_rate'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(0, config['epochs']):

        running_loss, running_accuracy, running_batch = 0,0,0
        model.train()
        for (x, y) in train_dataloader:
            optimizer.zero_grad()
            (x, y) = (x.to(device), y.to(device))
            pred = model(x)
            loss = criterion(pred, y)
            running_loss += loss.item()*x.size()[0]
            running_accuracy += (pred.argmax(1)==y).sum().item()
            running_batch +=y.size()[0]
            loss.backward()
            optimizer.step()
        training_loss.append(running_loss/len(train_dataloader.dataset))
        training_accuracy.append(100*(running_accuracy/running_batch))
        
        running_loss, running_accuracy, running_batch = 0,0,0
        with torch.no_grad():
            model.eval()
            for (x, y) in val_dataloader:
                (x, y) = (x.to(device), y.to(device))
                pred = model(x)
                loss = criterion(pred, y)
                running_loss += loss.item()*x.size()[0]
                running_accuracy += (pred.argmax(1)==y).sum().item()
                running_batch +=y.size()[0]

        validation_loss.append(running_loss/len(val_dataloader.dataset))
        validation_accuracy.append(100*(running_accuracy/running_batch))
        
        print(f"Epoch: {epoch+1}/{config['epochs']}")
        print(validation_loss)
        print(validation_accuracy)
        torch.cuda.empty_cache()

In [9]:
config = {
    'number_of_filters': 32,
    'filter_size': 3,
    'stride': 1,
    'padding': 1,
    'max_pooling_size': 2,
    'n_neurons': 256,
    'n_classes': 10,
    'conv_activation': 'relu',
    'dense_activation': 'relu',
    'dropout_rate': 0.2,
    'use_batchnorm': True,
    'factor':1,
    'learning_rate': 1e-3,
    'batch_size':16,
    'epochs':0,
    'use_augmentation': False,
    'dropout_organisation': 1
}

In [10]:
def wandb_sweep():
    
    run = wandb.init()
    config = wandb.config
    run.name = f"nf_{config['number_of_filters']}_fs_{config['filter_size']}_nn_{config['n_neurons']}_lr_{config['learning_rate']}_bs_{config['batch_size']}_cact_{config['conv_activation']}"

    training_data = ImageFolder(root=training_data_path, transform=get_transform(config['use_augmentation']))
    train_size = int(0.8 * len(training_data))
    val_size = len(training_data) - train_size
    train_set, validation_set = random_split(training_data, [train_size, val_size])
    train_dataloader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(validation_set, batch_size=config['batch_size'], shuffle=False)


    training_loss, training_accuracy, validation_loss, validation_accuracy = [],  [],  [], []

    activations = {
      'relu': nn.ReLU(),
      'gelu': nn.GELU(),
      'silu': nn.SiLU(),
      'mish': nn.Mish(),
      'relu6':nn.ReLU6(),
      'tanh': nn.Tanh(),
      'sigmoid': nn.Sigmoid()
    }
    gc.collect()
    torch.cuda.empty_cache()
    model = CNN(
      input_dimension=(3,224,224),
        number_of_filters=config['number_of_filters'],
        filter_size =(config['filter_size'],config['filter_size']),
        stride=config['stride'],
        padding=config['padding'],
        max_pooling_size=(config['max_pooling_size'],config['max_pooling_size']),
        n_neurons=config['n_neurons'],
        n_classes=config['n_classes'],
        conv_activation=activations[config['conv_activation']],
        dense_activation=activations[config['dense_activation']],
        dropout_rate=config['dropout_rate'],
        use_batchnorm=config['use_batchnorm'],
        factor=config['factor'],
        dropout_organisation=config['dropout_organisation']
    ).to(device)
    optimizer = Adam(model.parameters(), lr = config['learning_rate'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(0, config['epochs']):

        running_loss, running_accuracy, running_batch = 0,0,0
        model.train()
        for (x, y) in train_dataloader:
            optimizer.zero_grad()
            (x, y) = (x.to(device), y.to(device))
            pred = model(x)
            loss = criterion(pred, y)
            running_loss += loss.item()*x.size()[0]
            running_accuracy += (pred.argmax(1)==y).sum().item()
            running_batch +=y.size()[0]
            del x
            del y
            loss.backward()
            optimizer.step()
        training_loss.append(running_loss/len(train_dataloader.dataset))
        training_accuracy.append(100*(running_accuracy/running_batch))
        
        running_loss, running_accuracy, running_batch = 0,0,0
        with torch.no_grad():
            model.eval()
            for (x, y) in val_dataloader:
                (x, y) = (x.to(device), y.to(device))
                pred = model(x)
                loss = criterion(pred, y)
                running_loss += loss.item()*x.size()[0]
                running_accuracy += (pred.argmax(1)==y).sum().item()
                running_batch +=y.size()[0]
                del x
                del y
        validation_loss.append(running_loss/len(val_dataloader.dataset))
        validation_accuracy.append(100*(running_accuracy/running_batch))
        
        wandb.log({
                "epochs": epoch + 1,
                "train_loss": training_loss[-1],
                "train_accuracy": training_accuracy[-1],
                "val_loss": validation_loss[-1],
                "val_accuracy": validation_accuracy[-1]
        })
        print(f"Epoch: {epoch+1}/{config['epochs']}")
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
sweep_config = {
    'method': 'bayes',
    'name': 'PART_A_Q2_SWEEP_1',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize',
    },
    'parameters': {
        'number_of_filters': {
            'values': [16, 32, 64, 128, 256]
        },
        'filter_size': {
            'value':3
        },
        'stride': {
            'value':1
        },
        'padding': {
            'value':1
        },
        'max_pooling_size': {
            'value':2
        },
        'n_neurons': {
            'values':[64,128,256,512,1024]
        },
        'n_classes': {
            'value':10
        },
        'conv_activation': {
            'values':['relu','gelu','silu','mish','relu6','tanh','sigmoid']
        },
        'dense_activation': {
            'values':['relu','gelu','silu','mish','relu6','tanh','sigmoid']
        },
        'dropout_rate':{
            'values':[0.2,0.3,0.4,0.5]
        },
        'use_batchnorm':{
            'values':[True, False]
        },
        'factor':{
            'values':[1,2,3,0.5]
        },
        'learning_rate':{
            'values':[1e-2,1e-3,1e-4,1e-5]
        },
        'batch_size':{
            'value': 16
        },
        'epochs':{
            'values':[5,10,15,20]
        },
        'use_augmentation':{
            'values':[True, False]
        },
        'dropout_organisation':{
            'values':[1,2,3,4,5]
        }
    }
}

In [12]:
# Setup Wandb
wandb.login(key='API_KEY')

wandb: Currently logged in as: ed23s037. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/user/.netrc


True

In [ ]:
wandb.init(project="PROJECT", entity='ENTITY')

# Do Sweep
wandb_id = wandb.sweep(sweep_config, project="PROJECT")
wandb.agent(wandb_id, function=wandb_sweep, count=300)

# Finish
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: c6nq6gy5
Sweep URL: https://wandb.ai/ed23s037/CS6910_AS2/sweeps/c6nq6gy5
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: lltkk6a0 with config:
wandb: 	batch_size: 16
wandb: 	conv_activation: silu
wandb: 	dense_activation: relu6
wandb: 	dropout_organisation: 1
wandb: 	dropout_rate: 0.4
wandb: 	epochs: 5
wandb: 	factor: 0.5
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.01
wandb: 	max_pooling_size: 2
wandb: 	n_classes: 10
wandb: 	n_neurons: 1024
wandb: 	number_of_filters: 64
wandb: 	padding: 1
wandb: 	stride: 1
wandb: 	use_augmentation: True
wandb: 	use_batchnorm: False


Exception in thread ChkStopThrException in thread :
Exception in thread IntMsgThrNetStatThr:
Traceback (most recent call last):
:
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 982, in run
    self.run()
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/user/anaconda3/lib/python3.11/site-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status
    self._target(*self._args, **self._kwargs)
  File "/home/user/anaconda3/lib/python3.11/site-packages/wandb/sdk/wandb_run.py", line 268, in check_network_status
    self.run()
  File "/h

Epoch: 1/5
Epoch: 2/5
Epoch: 3/5
Epoch: 4/5
Epoch: 5/5


...

Epoch: 1/20
Epoch: 2/20
Epoch: 3/20
Epoch: 4/20


wandb: Network error (ConnectTimeout), entering retry loop.


Epoch: 5/20
Epoch: 6/20
Epoch: 7/20
Epoch: 8/20
Epoch: 9/20
Epoch: 10/20
Epoch: 11/20


wandb: Network error (ConnectTimeout), entering retry loop.


Epoch: 12/20
Epoch: 13/20
Epoch: 14/20
Epoch: 15/20
Epoch: 16/20
Epoch: 17/20
Epoch: 18/20
Epoch: 19/20
Epoch: 20/20
